In [1]:
def dp (capacity,item,weight,value,record):
    if record[capacity][item] != -1:
        return record[capacity][item]
    pick, unpick = 0,0
    if capacity>=weight[item]:
        pick = dp (capacity-weight[item],item-1,weight,value,record)
        pick += value[item]
    unpick = dp (capacity, item-1,weight,value,record)
    #print(capacity,item,pick,unpick)
    if pick>unpick:
        record[capacity][item] = pick 
        #backtrack[capacity][item] = 2
    else:
        record[capacity][item] = unpick
        #backtrack[capacity][item] = 1
    
    return record[capacity][item]

In [2]:
def back_track(capacity,item):
    ans = []
    while (backtrack[capacity][item]>0):        
        if backtrack[capacity][item] == 2:
            ans.append(item)
            capacity -= W[item]
        item -= 1
        #print(capacity,item)
        if capacity<0 or item<0:
            break
    return list(reversed(ans))
        
    

In [3]:
from random import randrange
import time

In [4]:
class BinomialTree:
    def __init__(self, key, value, sum = 0, value_sum = 0, read_key_or_sum = 0,order = 0):
        self.key = key
        self.value = value
        self.sum = key
        if sum != 0:
            self.sum = sum
        self.value_sum = value
        if value_sum != 0:
            self.value_sum = value_sum
        self.read_key_or_sum = read_key_or_sum
        self.children = []
        self.order = order
    def print_tree(self,low,high,d):
        #print("key:%i,%i,root:%i,value_sum:%i,read:%i"%(self.key,self.sum,self.order,self.value_sum,self.read_key_or_sum))
        if self.read_key_or_sum:
            '''
            if self.sum<high:
                print("sum:%i,value:%i"%(self.sum,self.value_sum))                
                if self.sum in d.keys():                  
                    d[self.sum] += [self.value_sum]
                else:
                    d.setdefault(self.sum,[self.value_sum])
            '''
            return
        else:
            #print("key:%i,%i,root:%i,children:%i"%(self.key,self.sum,self.order,len(self.children)))
            #print("key:%i,value:%i"%(self.key,self.value))                
            if self.key in d.keys():                  
                d[self.key] += [self.value]
            else:
                d.setdefault(self.key,[self.value])
            for child in self.children:
                #print("child key:%i,%i,root:%i,children:%i"%(child.key,child.sum,child.order,len(child.children)))
                #print(child.children[0],child)
                child.print_tree(low,high,d)
    def find_tree_to_merge(self,merge_heap):
        if self.read_key_or_sum == 1:
            return
        if self.sum>=merge_heap.low and self.sum<merge_heap.high:
            self.read_key_or_sum = 1
            print("merge tree:%i"%self.sum)
            #tmp_tree = BinomialTree(self.key,self.value,self.sum,self.value_sum,self.read_key_or_sum,self.order)
            tmp_tree = BinomialTree(self.sum,self.value_sum)
            #tmp_tree.children = self.children
            #tmp_tree.read_key_or_sum = 1
            merge_heap.trees.append(tmp_tree)
            return
        else:
            for child in self.children:
                if child.sum>=merge_heap.low and child.sum<merge_heap.high:
                    child.read_key_or_sum = 1
                    print("merge child:%i"%child.sum)
                    #tmp_tree = BinomialTree(child.key,child.value,child.sum,child.value_sum,child.read_key_or_sum,child.order)
                    tmp_tree = BinomialTree(child.sum,child.value_sum)
                    #tmp_tree.children = child.children
                    #tmp_tree.read_key_or_sum = 1
                    merge_heap.trees.append(tmp_tree)
                elif child.read_key_or_sum==0:
                    child.find_tree_to_merge(merge_heap)
    def add_at_end(self, t):
        self.children.append(t)
        self.sum += t.sum
        self.value_sum += t.value_sum
        self.order = self.order + 1

In [5]:
class BinomialHeap:
    def __init__(self,low,high):
        self.trees = []
        self.low = low
        self.high = high
        self.dict = ()
 
    def extract_max(self):
        if self.trees == []:
            return None
        largest_node = self.trees[0]
        for tree in self.trees:
            if tree.key > largest_node.key:
                largest_node = tree
        self.trees.remove(largest_node)
        h = BinomialHeap(self.low,self.high)
        h.trees = largest_node.children
        self.merge(h)
 
        return largest_node.key
 
    def get_max(self):
        if self.trees == []:
            return None
        mx = self.trees[0].key
        for tree in self.trees:
            if tree.key > mx:
                mx = tree.key
        return mx

    def find_trees_to_merge(self,t_next):       
        if self.trees == []:
            return None
        tmp_heap = BinomialHeap(t_next.low,t_next.high)
        for tree in self.trees:
            tree.find_tree_to_merge(tmp_heap)
        t_next.merge(tmp_heap)
                        
    def print_heap(self):
        self.d = dict()
        if self.trees == []:
            print("Empty!")
            return
        for tree in self.trees:
            tree.print_tree(self.low,self.high,self.d)
    
        
    def combine_roots(self, h):
        self.trees.extend(h.trees)
        self.trees.sort(key=lambda tree: tree.order)
 
    def merge(self, h):
        self.combine_roots(h)
        if self.trees == []:
            return
        i = 0
        while i < len(self.trees) - 1:
            current = self.trees[i]
            after = self.trees[i + 1]
            if current.order == after.order:
                if (i + 1 < len(self.trees) - 1
                    and self.trees[i + 2].order == after.order):
                    after_after = self.trees[i + 2]
                    if after.key > after_after.key:
                        after.add_at_end(after_after)
                        del self.trees[i + 2]
                    else:
                        after_after.add_at_end(after)
                        del self.trees[i + 1]
                else:
                    if current.key > after.key:
                        current.add_at_end(after)
                        del self.trees[i + 1]
                    else:
                        after.add_at_end(current)
                        del self.trees[i]
            i = i + 1
 
    def insert(self, key,value):
        g = BinomialHeap(self.low,self.high)
        g.trees.append(BinomialTree(key,value))
        self.merge(g)

In [6]:
num = 50
interval = 20
base = 10
C = 5000
W = [-1]
W_1 = [-1]
w = dict()
h = dict()
for j in range (interval):
    low = j*base
    high = (j+1)*base
    h.setdefault(low,BinomialHeap(low,high))
for j in range (interval):    
    low = j*base
    high = (j+1)*base
    print("%i,%i:"%(low,high))
    w.setdefault(low, {'w':[randrange(max(1,low),high) for i in range (num)],\
                       'v':[randrange(1,50) for i in range (num)]})
    print('w:',w[low]['w'])
    print('v:',w[low]['v'])
    for a,b in zip(w[low]['w'],w[low]['v']):
        h[low].insert(a,b)

0,10:
w: [5, 1, 6, 4, 8, 2, 3, 8, 3, 9, 4, 7, 2, 8, 1, 3, 1, 6, 1, 1, 6, 7, 5, 8, 4, 8, 9, 2, 4, 6, 4, 3, 1, 2, 6, 5, 3, 8, 8, 2, 7, 6, 7, 3, 1, 7, 9, 1, 6, 3]
v: [2, 16, 34, 44, 47, 35, 21, 14, 40, 12, 33, 5, 37, 22, 47, 2, 2, 18, 17, 48, 47, 1, 6, 29, 34, 32, 16, 9, 34, 6, 7, 5, 6, 29, 31, 44, 27, 39, 27, 45, 16, 30, 11, 47, 47, 16, 11, 4, 16, 17]
10,20:
w: [17, 11, 10, 15, 17, 16, 11, 16, 12, 15, 17, 18, 10, 12, 17, 19, 15, 15, 14, 10, 10, 11, 16, 13, 10, 10, 11, 14, 11, 19, 13, 18, 18, 16, 16, 13, 19, 12, 17, 18, 18, 15, 19, 13, 10, 12, 11, 19, 13, 13]
v: [4, 25, 31, 8, 24, 10, 21, 24, 18, 47, 18, 1, 19, 46, 41, 42, 26, 20, 14, 26, 38, 38, 48, 38, 39, 35, 30, 19, 39, 36, 47, 49, 30, 39, 18, 40, 1, 40, 32, 1, 45, 10, 47, 14, 9, 30, 18, 20, 30, 5]
20,30:
w: [22, 28, 28, 21, 21, 28, 28, 29, 26, 22, 22, 24, 21, 21, 21, 23, 25, 29, 27, 20, 24, 22, 24, 27, 23, 25, 24, 20, 25, 23, 22, 22, 24, 24, 21, 29, 22, 21, 23, 28, 20, 26, 21, 27, 21, 22, 22, 28, 20, 25]
v: [4, 30, 12, 46, 25, 9, 33,

In [7]:
for j in range(interval):
    print(j)
    low = j*base
    h[low].print_heap()
    '''
    for key,value in h[low].d.items():
        print("(%i,"%key,value,")",end='')
    print()
    '''
    for k in range(interval-1,j,-1):
        h[low].find_trees_to_merge(h[k*base])
    #print("Max1:%i"%h[low].get_max())
    h[low].print_heap()
    for key,value in h[low].d.items():
        print("(%i,"%key,value,")",end='')
    print()

0
merge tree:149
merge tree:41
merge tree:35
(6, [16] )(3, [17] )
1
merge child:117
merge child:113
merge child:104
merge child:63
merge child:62
merge child:45
merge child:35
merge child:31
merge tree:26
merge child:22
(19, [1, 42, 36] )(12, [40] )(17, [41] )(11, [39] )
2
merge child:187
merge child:180
merge child:184
merge child:94
merge child:99
merge child:97
merge tree:48
merge tree:45
merge child:48
merge child:49
merge child:47
(29, [7, 30, 41] )(21, [27] )(28, [33] )(25, [13] )
3
merge child:146
merge tree:137
merge child:134
merge child:139
merge child:137
merge child:128
merge child:126
merge child:72
merge child:71
merge child:65
merge child:67
merge child:60
merge child:69
(35, [248, 34] )(39, [25, 33, 45, 40] )(37, [20, 37] )(38, [14] )(30, [11] )(31, [8] )(33, [32, 4] )
4
merge tree:189
merge child:188
merge child:182
merge tree:179
merge child:177
merge child:179
merge child:178
merge child:174
merge child:93
merge child:91
merge child:94
merge child:88
merge child:85
m

In [8]:
W = [-1]
W_1 = [-1]
W_2 = [-1]
V = [-1]
V_1 = [-1]
for j in range (interval):
    W += w[j*base]['w']
    heap = h[j*base].d
    for k,vs in heap.items():
        W_1 += [k]*len(vs)
        W_2 += [j*base+base//2]*len(vs)
        V_1 += vs
    #W_2 += [j*base+base//2 for i in range (num)]
    V += w[j*base]['v']
assert(len(W_1)==len(V_1))
record = [[-1]*(num*interval+1) for i in range(C+1)]
record_1 = [[-1]*(len(W_1)) for i in range(C+1)]
record_2 = [[-1]*(len(W_1)) for i in range(C+1)]
backtrack = [[-1]*(num*interval+1) for i in range(C+1)]
for i in range(C+1):
    record[i][0] = 0
    record_1[i][0] = 0
    record_2[i][0] = 0
for i in range(num*interval+1):
    record[0][i] = 0
for i in range(len(W_1)):
    record_1[0][i] = 0
    record_2[0][i] = 0
#for w,v in zip (W[1:],W_1[1:]):
#    print("(%02i, %02i) "%(w,v),end="")

In [9]:
print("NUM before compression:%i,after compression:%i"%(num*interval,len(W_1)-1))
print(sum(V[1:]),sum(V_1[1:]))
assert(sum(V[1:])==sum(V_1[1:]))
#print(W_1,W_2)
now = time.time()
print("Ans:",dp (C,num*interval,W,V,record))
print(time.time()-now)
now = time.time()
print("Ans1:",dp (C,len(W_1)-1,W_1,V_1,record_1))
print(time.time()-now)
now = time.time()
print("Ans2:",dp (C,len(W_2)-1,W_2,V_1,record_2))
print(time.time()-now)
#print(back_track(C,num*interval))
#for i in backtrack:
#    print(i)

NUM before compression:1000,after compression:762
24752 24752
Ans: 6439
3.24853777885437
Ans1: 5983
2.355226993560791
Ans2: 6020
0.373380184173584
